# PPO for Marketing Budget Optimization
## Multi-Channel Marketing with Proximal Policy Optimization

This notebook demonstrates how to use Proximal Policy Optimization (PPO) for optimizing marketing budget allocation across multiple channels.

### Learning Objectives:
- Understand PPO algorithm and policy gradient methods
- Implement multi-channel marketing environment
- Optimize budget allocation for maximum ROI
- Analyze marketing performance metrics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
from collections import deque
import random

np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

print("✅ Libraries imported successfully!")
print("📈 Ready to optimize marketing with PPO!")

## 1. Marketing Environment Implementation

Multi-channel marketing environment with realistic business constraints and synergies.

In [ ]:
class MarketingEnvironment:
    def __init__(self, total_budget=10000, num_channels=4):
        self.total_budget = total_budget
        self.num_channels = num_channels
        self.channel_names = ['Social Media', 'Search', 'Display', 'Email']
        
        # Channel characteristics based on industry data
        self.channel_params = {
            'effectiveness': np.array([0.8, 1.2, 0.6, 0.9]),
            'saturation': np.array([3000, 4000, 2000, 1500]),
            'efficiency': np.array([0.15, 0.25, 0.10, 0.20]),
            'synergy_matrix': np.array([
                [1.0, 0.1, 0.05, 0.15],
                [0.1, 1.0, 0.08, 0.12],
                [0.05, 0.08, 1.0, 0.06],
                [0.15, 0.12, 0.06, 1.0]
            ])
        }
        self.reset()
    
    def reset(self):
        self.time_step = 0
        self.remaining_budget = self.total_budget
        self.seasonality = np.random.uniform(0.8, 1.2, self.num_channels)
        self.competition = np.random.uniform(0.9, 1.1, self.num_channels)
        self.market_trend = np.random.uniform(-0.1, 0.1)
        self.performance_history = deque(maxlen=10)
        return self._get_state()
    
    def _get_state(self):
        state = np.concatenate([
            [self.remaining_budget / self.total_budget],
            [self.time_step / 52],
            self.seasonality,
            self.competition,
            [self.market_trend],
            [np.mean([p['roi'] for p in self.performance_history]) if self.performance_history else 0],
            [np.std([p['roi'] for p in self.performance_history]) if len(self.performance_history) > 1 else 0],
        ])
        return state
    
    def step(self, action):
        allocation = action * self.remaining_budget
        conversions = self._calculate_conversions(allocation)
        
        revenue = np.sum(conversions) * 50  # $50 AOV
        cost = np.sum(allocation)
        roi = (revenue - cost) / cost if cost > 0 else 0
        profit = revenue - cost
        
        self.remaining_budget -= cost
        self.time_step += 1
        
        performance = {'roi': roi, 'profit': profit, 'revenue': revenue}
        self.performance_history.append(performance)
        
        reward = self._calculate_reward(performance, allocation)
        self._update_market_conditions()
        
        done = self.remaining_budget < 100 or self.time_step >= 52
        
        info = {
            'conversions': conversions,
            'revenue': revenue,
            'roi': roi,
            'profit': profit,
            'allocation': allocation
        }
        
        return self._get_state(), reward, done, info
    
    def _calculate_conversions(self, allocation):
        conversions = np.zeros(self.num_channels)
        for i in range(self.num_channels):
            if allocation[i] > 0:
                saturation = 1 - np.exp(-allocation[i] / self.channel_params['saturation'][i])
                base = allocation[i] * self.channel_params['effectiveness'][i] * saturation
                conversions[i] = base * self.seasonality[i] / self.competition[i]
        
        # Apply synergies
        for i in range(self.num_channels):
            for j in range(self.num_channels):
                if i != j and allocation[j] > 0:
                    synergy = conversions[j] * self.channel_params['synergy_matrix'][i][j]
                    conversions[i] += synergy * allocation[i] / self.total_budget
        
        return np.maximum(0, conversions + np.random.normal(0, 0.1 * conversions))
    
    def _calculate_reward(self, performance, allocation):
        roi_reward = performance['roi'] * 10
        
        # Diversity bonus
        allocation_norm = allocation / np.sum(allocation)
        entropy = -np.sum(allocation_norm * np.log(allocation_norm + 1e-8))
        diversity_bonus = entropy * 2
        
        # Penalty for extreme allocation
        max_ratio = np.max(allocation) / np.sum(allocation)
        extreme_penalty = -5 if max_ratio > 0.8 else 0
        
        return roi_reward + diversity_bonus + extreme_penalty
    
    def _update_market_conditions(self):
        # Seasonal changes
        seasonal_change = 0.1 * np.sin(2 * np.pi * self.time_step / 52)
        self.seasonality = np.clip(self.seasonality + seasonal_change * 0.1, 0.5, 1.5)
        
        # Competition changes
        self.competition += np.random.normal(0, 0.02, self.num_channels)
        self.competition = np.clip(self.competition, 0.8, 1.3)
        
        # Market trend
        self.market_trend += np.random.normal(0, 0.01)
        self.market_trend = np.clip(self.market_trend, -0.2, 0.2)

# Test environment
env = MarketingEnvironment()
print(f"📊 Marketing Environment Created!")
print(f"Channels: {env.channel_names}")
print(f"State dimension: {len(env.reset())}")

## 2. PPO Agent Implementation

In [ ]:
class PPONetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=128):
        super(PPONetwork, self).__init__()
        self.shared_fc1 = nn.Linear(state_size, hidden_size)
        self.shared_fc2 = nn.Linear(hidden_size, hidden_size)
        self.actor_fc = nn.Linear(hidden_size, hidden_size)
        self.actor_out = nn.Linear(hidden_size, action_size)
        self.critic_fc = nn.Linear(hidden_size, hidden_size)
        self.critic_out = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, x):
        x = F.relu(self.shared_fc1(x))
        x = self.dropout(x)
        x = F.relu(self.shared_fc2(x))
        
        # Actor
        actor_x = F.relu(self.actor_fc(x))
        action_logits = self.actor_out(actor_x)
        action_probs = F.softmax(action_logits, dim=-1)
        
        # Critic
        critic_x = F.relu(self.critic_fc(x))
        state_value = self.critic_out(critic_x)
        
        return action_probs, state_value

class PPOAgent:
    def __init__(self, state_size, action_size, lr=3e-4, gamma=0.99, epsilon=0.2):
        self.network = PPONetwork(state_size, action_size)
        self.optimizer = optim.Adam(self.network.parameters(), lr=lr)
        self.gamma = gamma
        self.epsilon = epsilon
        
        # Storage
        self.states, self.actions, self.rewards = [], [], []
        self.values, self.log_probs, self.dones = [], [], []
        
        # Metrics
        self.episode_rewards = []
        self.episode_rois = []
    
    def get_action(self, state):
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            action_probs, state_value = self.network(state_tensor)
        
        dist = Categorical(action_probs)
        action_idx = dist.sample()
        allocation = action_probs.squeeze().numpy()
        
        return allocation, action_idx.item(), dist.log_prob(action_idx).item(), state_value.item()
    
    def store_transition(self, state, action, reward, value, log_prob, done):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.values.append(value)
        self.log_probs.append(log_prob)
        self.dones.append(done)
    
    def update(self):
        if not self.states:
            return
        
        # Compute returns
        returns = []
        R = 0
        for i in reversed(range(len(self.rewards))):
            R = self.rewards[i] + self.gamma * R * (1 - self.dones[i])
            returns.insert(0, R)
        
        # Convert to tensors
        states = torch.FloatTensor(self.states)
        actions = torch.LongTensor(self.actions)
        old_log_probs = torch.FloatTensor(self.log_probs)
        returns = torch.FloatTensor(returns)
        advantages = returns - torch.FloatTensor(self.values)
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        # PPO update
        for _ in range(4):
            action_probs, values = self.network(states)
            dist = Categorical(action_probs)
            new_log_probs = dist.log_prob(actions)
            
            ratio = torch.exp(new_log_probs - old_log_probs)
            surr1 = ratio * advantages
            surr2 = torch.clamp(ratio, 1-self.epsilon, 1+self.epsilon) * advantages
            
            policy_loss = -torch.min(surr1, surr2).mean()
            value_loss = F.mse_loss(values.squeeze(), returns)
            entropy = dist.entropy().mean()
            
            total_loss = policy_loss + 0.5 * value_loss - 0.01 * entropy
            
            self.optimizer.zero_grad()
            total_loss.backward()
            self.optimizer.step()
        
        self.clear_storage()
    
    def clear_storage(self):
        self.states, self.actions, self.rewards = [], [], []
        self.values, self.log_probs, self.dones = [], [], []

# Create agent
state_size = len(env.reset())
agent = PPOAgent(state_size, env.num_channels)
print(f"🤖 PPO Agent Created! State size: {state_size}")

## 3. Training and Results

In [ ]:
# Training function
def train_ppo_agent(agent, env, episodes=150):
    for episode in range(episodes):
        state = env.reset()
        episode_reward = 0
        episode_roi = 0
        steps = 0
        
        while True:
            allocation, action_idx, log_prob, value = agent.get_action(state)
            next_state, reward, done, info = env.step(allocation)
            
            agent.store_transition(state, action_idx, reward, value, log_prob, done)
            
            state = next_state
            episode_reward += reward
            episode_roi += info['roi']
            steps += 1
            
            if done:
                break
        
        agent.episode_rewards.append(episode_reward)
        agent.episode_rois.append(episode_roi / steps if steps > 0 else 0)
        
        if (episode + 1) % 20 == 0:
            agent.update()
        
        if (episode + 1) % 25 == 0:
            avg_reward = np.mean(agent.episode_rewards[-25:])
            avg_roi = np.mean(agent.episode_rois[-25:])
            print(f"Episode {episode + 1:3d} | Reward: {avg_reward:6.2f} | ROI: {avg_roi:5.2f}")

# Train
print("🚀 Training PPO Agent...")
train_ppo_agent(agent, env)
print("✅ Training Complete!")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Learning curves
axes[0].plot(agent.episode_rewards, alpha=0.6, label='Episode Reward')
if len(agent.episode_rewards) >= 20:
    moving_avg = pd.Series(agent.episode_rewards).rolling(20).mean()
    axes[0].plot(moving_avg, 'r-', linewidth=2, label='20-episode average')
axes[0].set_title('Training Progress')
axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Reward')
axes[0].legend()
axes[0].grid(True)

# ROI progression
axes[1].plot(agent.episode_rois, alpha=0.6, label='Episode ROI')
if len(agent.episode_rois) >= 20:
    roi_avg = pd.Series(agent.episode_rois).rolling(20).mean()
    axes[1].plot(roi_avg, 'g-', linewidth=2, label='20-episode average')
axes[1].set_title('ROI Performance')
axes[1].set_xlabel('Episode')
axes[1].set_ylabel('ROI')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print(f"\n📊 Final Performance:")
print(f"   • Average ROI: {np.mean(agent.episode_rois[-20:]):.2f}")
print(f"   • Training completed successfully!")